In [22]:
import cv2
import numpy as np

fold_path = './data/msnn4_fold5'
model = 'mcnn4'
out_img_ori = "ucf-cc-50-gt.png"
out_img_model = "ucf-cc-50-et.png"
image = "./data/utils/ucf-cc-50/26.jpg"
den_map = "./data/utils/ucf-cc-50/26.npy"
alpha = 0.35
den_scale_factor = 1e3

In [24]:
from architecture.crowd_count import CrowdCounter
import architecture.network as network

import os.path as osp

# gt
img = cv2.imread(image, cv2.IMREAD_COLOR)
den = np.load(den_map)

den = (den/np.max(den) * 255).astype("uint8")
heatmap = cv2.applyColorMap(den, cv2.COLORMAP_JET)
super_img = (1-alpha) * img + alpha * heatmap

cv2.imwrite(out_img_ori, super_img)

#model
imgt = cv2.imread(image, 0)
net = CrowdCounter(model = model).cuda().eval()
pretrained_model = osp.join(fold_path, 'best_model.h5')
network.load_net(pretrained_model, net)

imgt = imgt.astype(np.float32, copy=False)
imgt = imgt.reshape((1,imgt.shape[0],imgt.shape[1]))
imgt = imgt / 127.5 - 1 
imgt = np.array([imgt])
#imgt = network.np_to_variable(imgt, is_cuda=True, is_training=False)
den = net(imgt)

den = den.data.cpu().numpy()
den /= den_scale_factor
den = den.reshape((den.shape[2], den.shape[3]))

ht = den.shape[0]
wd = den.shape[1]
wd_1 = img.shape[1]
ht_1 = img.shape[0]
den = cv2.resize(den,(wd_1,ht_1))
den = den * ((wd*ht)/(wd_1*ht_1)) #fix people count

print(img.shape, den.shape, imgt.shape)

den = (den/np.max(den) * 255).astype("uint8")
heatmap = cv2.applyColorMap(den, cv2.COLORMAP_JET)
super_img = (1-alpha) * img + alpha * heatmap

cv2.imwrite(out_img_model, super_img)





(712, 1024, 3) (1024, 712) (1, 1, 712, 1024)


ValueError: operands could not be broadcast together with shapes (712,1024,3) (1024,712,3) 